In [1]:
def levdist(s, t):
    '''function to calculate the
    Levenshtein distance between
    two strings in a recursive way'''
    
    if s == '':
        return len(t)
    if t == '':
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
       
    dist = min([levdist(s[:-1], t)+1,
               levdist(s, t[:-1])+1, 
               levdist(s[:-1], t[:-1]) + cost])

    return dist

In [2]:
levdist('irks', 'risk')

3

In [3]:
import time

st = time.time()
s1 = 'micrsft'
s2 = 'microsoft corporation'
print('Levenshtein distance between ',
      f'{s1}', ' and ', f'{s2}', ' is:', levdist(s1, s2))
et = time.time()
print('time taken to calculate Levenshtein distance between ',
      f'{s1}', ' and ', f'{s2}', ' is:', round(et-st, 2), 's')

Levenshtein distance between  micrsft  and  microsoft corporation  is: 14
time taken to calculate Levenshtein distance between  micrsft  and  microsoft corporation  is: 42.93 s


This is slow!
Iterative computation using matrix improves the computation time

In [4]:
def iterative_levdist(s, t):
    '''function to calculate the
    Levenshtein distance between
    two strings in an iterative way'''

    rows = len(s)+1
    cols = len(t)+1
    dist = [[0 for x in range(cols)] for x in range(rows)]

    # source prefixes can be transformed into empty strings 
    # by deletions:
    for i in range(1, rows):
        dist[i][0] = i

    # target prefixes can be created from an empty source string
    # by inserting the characters
    for i in range(1, cols):
        dist[0][i] = i
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = 1
            dist[row][col] = min(dist[row-1][col] + 1,      # deletion
                                 dist[row][col-1] + 1,      # insertion
                                 dist[row-1][col-1] + cost) # substitution

 
    return dist[row][col]

In [5]:
iterative_levdist('irks', 'risk')

3

In [6]:
st = time.time()
s1 = 'micrsft'
s2 = 'microsoft corporation'
print('Levenshtein distance between ',
      f'{s1}', ' and ', f'{s2}', ' is:', iterative_levdist(s1, s2))
et = time.time()
print('time taken to calculate Levenshtein distance between ',
      f'{s1}', ' and ', f'{s2}', ' is:', round(et-st, 2), 's')

Levenshtein distance between  micrsft  and  microsoft corporation  is: 14
time taken to calculate Levenshtein distance between  micrsft  and  microsoft corporation  is: 0.0 s


Damerau-Levenshtein Distance works similar to Levenshtein distance

To install jellyfish package, pip install jellyfish

In [7]:
!pip install jellyfish

In [8]:
import jellyfish

print('Levenshtein distance is: ', jellyfish.levenshtein_distance('irks', 'risk'))
print('Damerau-Levenshtein distance is: ', jellyfish.damerau_levenshtein_distance('irks', 'risk'))

Levenshtein distance is:  3
Damerau-Levenshtein distance is:  2


In [9]:
import sys

def bitap_search(text, pattern):
    
    '''function to do a bit-ap search
    using bit array
    (this is python implementation of the
    code in GeekforGeeks)'''
    
    len_pattern = len(pattern)
    
    #initializing the bit array to complement of 0
    bit_array = [~0] * (sys.maxunicode)
    
    # R is a variable, initiliazing it to complement of 1
    R = ~1
    
    # taking care of the edge cases
    # when the pattern is absent
    # or when the pattern is longer than the text
    if len_pattern == 0:
        return -1

    if len_pattern > len(text):
        print('Pattern too long!')
        return -1

    
    for i in range(len_pattern):
        bit_array[ord(pattern[i])] &= ~(1 << i)

    for i in range(len(text)):
        R |= bit_array[ord(text[i])]
        R <<= 1;
        if (R & (1 << len_pattern)) == 0:
            return i - len_pattern + 1

    return -1



def findPattern(t, p):
    text = list(t);
    pattern = list(p);
    index = bitap_search(text, pattern);
    if index == -1:
        print('No Match')
    else:
        print('Pattern found at index:', index)




In [10]:
text = 'womenwhocode'
pattern = 'code'

findPattern(text, pattern)

Pattern found at index: 8


In [11]:
text = 'youareamazing'
pattern = 'youareawesome'

findPattern(text, pattern)

No Match


It is no match because there is no substring. 

N-GRAM ALGORITHM

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = ['company name is microsoft', 'company the name is Microsoft', 
          'company name mcrosft', 'the company is Microsft Co',
         'company is Microsoft Corporation', 'the company is name microsoft Corp', 
          'company MCSFT CO name']

clean_corpus = ['The company name is Microsoft Corporation']

In [13]:
vect1 = TfidfVectorizer(analyzer='word', ngram_range=(1, 1))
x = vect1.fit_transform(corpus)

print(vect1.get_feature_names())

['co', 'company', 'corp', 'corporation', 'is', 'mcrosft', 'mcsft', 'microsft', 'microsoft', 'name', 'the']


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
vect2 = TfidfVectorizer(analyzer='word', ngram_range=(3, 3))
x = vect2.fit_transform(corpus)

print(vect2.get_feature_names())

['company is microsft', 'company is microsoft', 'company is name', 'company mcsft co', 'company name is', 'company name mcrosft', 'company the name', 'is microsft co', 'is microsoft corporation', 'is name microsoft', 'mcsft co name', 'name is microsoft', 'name microsoft corp', 'the company is', 'the name is']


In [15]:
vect3 = TfidfVectorizer(analyzer='word', ngram_range=(2, 5))
x = vect3.fit_transform(corpus)

print(vect3.get_feature_names())

['co name', 'company is', 'company is microsft', 'company is microsft co', 'company is microsoft', 'company is microsoft corporation', 'company is name', 'company is name microsoft', 'company is name microsoft corp', 'company mcsft', 'company mcsft co', 'company mcsft co name', 'company name', 'company name is', 'company name is microsoft', 'company name mcrosft', 'company the', 'company the name', 'company the name is', 'company the name is microsoft', 'is microsft', 'is microsft co', 'is microsoft', 'is microsoft corporation', 'is name', 'is name microsoft', 'is name microsoft corp', 'mcsft co', 'mcsft co name', 'microsft co', 'microsoft corp', 'microsoft corporation', 'name is', 'name is microsoft', 'name mcrosft', 'name microsoft', 'name microsoft corp', 'the company', 'the company is', 'the company is microsft', 'the company is microsft co', 'the company is name', 'the company is name microsoft', 'the name', 'the name is', 'the name is microsoft']


Let's implement on this example and compare with Levenshtein distance

In [16]:
!pip install fuzzy-match

In [17]:
import pandas as pd
import itertools

# for fuzzy matching
from fuzzy_match import algorithims

In [18]:
# make a cartesian combination of the words in the corpus and the correct word

cart_list = list(itertools.product(corpus, clean_corpus))
cart_list

[('company name is microsoft', 'The company name is Microsoft Corporation'),
 ('company the name is Microsoft',
  'The company name is Microsoft Corporation'),
 ('company name mcrosft', 'The company name is Microsoft Corporation'),
 ('the company is Microsft Co', 'The company name is Microsoft Corporation'),
 ('company is Microsoft Corporation',
  'The company name is Microsoft Corporation'),
 ('the company is name microsoft Corp',
  'The company name is Microsoft Corporation'),
 ('company MCSFT CO name', 'The company name is Microsoft Corporation')]

In [19]:
def get_matches(lst):
    
    '''function to calculate the Levenshtein
    distance, trigram match score and the cosine
    similarity matches for the above example'''
    
    x, y, lscore, tscore, cscore = [], [], [], [], []

    for i in range(len(lst)):
        x.append(lst[i][0])
        y.append(lst[i][1])
        
        lscore.append(round(algorithims.levenshtein(lst[i][0], lst[i][1]), 3))
        tscore.append(round(algorithims.trigram(lst[i][0], lst[i][1]), 3))
        cscore.append(round(algorithims.cosine(lst[i][0], lst[i][1]), 3))
    
    df = pd.DataFrame(list(zip(x, y, lscore, tscore, cscore)),
                    columns = ['in_data', 'clean_data', 'lev_score', 'tri_score', 'cosine_score'])
    
    return df

In [20]:
get_matches(cart_list)

,in_data,clean_data,lev_score,tri_score,cosine_score
0,company name is microsoft,The company name is Microsoft Corporation,0.585,0.650,0.612
1,company the name is Microsoft,The company name is Microsoft Corporation,0.512,0.750,0.730
2,company name mcrosft,The company name is Microsoft Corporation,0.463,0.386,0.471
3,the company is Microsft Co,The company name is Microsoft Corporation,0.610,0.512,0.365
4,company is Microsoft Corporation,The company name is Microsoft Corporation,0.780,0.775,0.816
5,the company is name microsoft Corp,The company name is Microsoft Corporation,0.634,0.780,0.500
6,company MCSFT CO name,The company name is Microsoft Corporation,0.293,0.333,0.408


IMPLEMENTATION ON REAL DATA

In [21]:
!pip install fuzzywuzzy

In [22]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\PALAK BHATT\AppData\Roaming\Python\Python39\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [23]:
# import libraries
import pandas as pd

# this is for fuzzy matching
from fuzzywuzzy import fuzz
from fuzzy_match import algorithims

In [28]:
data_df = pd.read_csv('D:\\room_type.csv')

In [29]:
data_df.head()

,Expedia,Booking.com
0,"Deluxe Room, 1 King Bed",Deluxe King Room
1,"Standard Room, 1 King Bed, Accessible",Standard King Roll-in Shower Accessible
2,"Grand Corner King Room, 1 King Bed",Grand Corner King Room
3,"Suite, 1 King Bed (Parlor)",King Parlor Suite
4,"High-Floor Premium Room, 1 King Bed",High-Floor Premium King Room


RATIO -Compares the entire string similarity

In [30]:
def get_ratio(row):
    
    '''function to compare the values in each row
    for the two columns in the same dataframe and
    return the ratio for the entire string similarity'''
    
    col1 = row['Expedia']
    col2 = row['Booking.com']
    
    return fuzz.ratio(col1, col2)

PARTIAL RATIO - Compares partial string similarity

In [31]:
def get_partial_ratio(row):
    
    '''function to compare the values in each row
    for the two columns in the same dataframeand
    return the ratio for partial string similarity'''
    
    col1 = row['Expedia']
    col2 = row['Booking.com']
    
    return fuzz.partial_ratio(col1, col2)

TOKEN SORT RATIO - Ignores word order

In [32]:
def get_token_sort_ratio(row):
    
    '''function to compare the values in each row
    for the two columns in the same dataframeand
    return the ratio for string similarity by
    ignoring word order'''
    
    col1 = row['Expedia']
    col2 = row['Booking.com']
    
    return fuzz.token_sort_ratio(col1, col2)

TOKEN SET RATIO - Ignore duplicate words similarly to token sort ratio

In [33]:
def get_token_set_ratio(row):
    
    '''function to compare the values in each row
    for the two columns in the same dataframeand
    return the ratio for string similarity by
    ignoring duplicate words and word order'''
    
    col1 = row['Expedia']
    col2 = row['Booking.com']
    
    return fuzz.token_set_ratio(col1, col2)

TRIGRAM - Calculates a similarity score and find matches by splitting strings into ngrams with a length of 3. The length of the ngram can be altered if desired.

In [34]:
def get_trigram_value(row):
    
    '''function to compare the values in each row
    for the two columns in the same dataframeand
    return the ratio for string similarity by
    ignoring duplicate words and word order'''
    
    col1 = row['Expedia']
    col2 = row['Booking.com']
    
    return round(algorithims.trigram(col1, col2), 3)

In [35]:
data_df['full_ratio'] = data_df.apply(get_ratio, axis=1)
data_df['partial_ratio'] = data_df.apply(get_partial_ratio, axis=1)
data_df['token_sort_ratio'] = data_df.apply(get_token_sort_ratio, axis=1)
data_df['token_set_ratio'] = data_df.apply(get_token_set_ratio, axis=1)
data_df['trigram'] = data_df.apply(get_trigram_value, axis=1)
data_df.head()

,Expedia,Booking.com,full_ratio,partial_ratio,token_sort_ratio,token_set_ratio,trigram
0,"Deluxe Room, 1 King Bed",Deluxe King Room,62,69,84,100,0.739
1,"Standard Room, 1 King Bed, Accessible",Standard King Roll-in Shower Accessible,68,65,78,81,0.562
2,"Grand Corner King Room, 1 King Bed",Grand Corner King Room,79,100,80,100,0.793
3,"Suite, 1 King Bed (Parlor)",King Parlor Suite,51,65,85,100,0.750
4,"High-Floor Premium Room, 1 King Bed",High-Floor Premium King Room,76,82,90,100,0.829


It looks like TOKEN SET RATIO from fuzzywuzzy package is the best method to get the most similar matches in this example.